In [1]:
%pip install boto3 pandas plotly requests pillow tabulate kaleido nbformat ipython

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import requests
import os

from utils import get_df_for_benchmark, performance_score, shorten_gpu_name, get_gpu_classes

In [2]:
data_cache_dir = "./data"
os.makedirs(data_cache_dir, exist_ok=True)
full_df_path = os.path.join(data_cache_dir, "all-data.csv")

def load_benchmark(benchmark_id: str):
    cache_location = os.path.join(data_cache_dir, f"{benchmark_id}.csv")
    df = None
    try:
        print(f"Loading data from cached CSV: {cache_location}")
        df = pd.read_csv(cache_location)
        df["timestamp"] = pd.to_datetime(df["timestamp"])
    except FileNotFoundError:
        # otherwise, get the data from the API
        print("Cached CSV not found. Loading data from DynamoDB")
        df = get_df_for_benchmark(benchmark_id)
        if df is not None:
            df["gpu"] = df["gpu"].apply(shorten_gpu_name)
            df["gpu_performance_score"] = df["gpu"].apply(performance_score)
            df.to_csv(cache_location, index=False)
    except Exception as e:
        print("Error loading data from CSV.")
        print(e)
    if df is not None:
        print(f"Loaded {len(df)} rows of data for benchmark {benchmark_id}")
    else:
        print(f"No data found for benchmark {benchmark_id}")
    return df


with open("benchmark_ids.txt", "r") as f:
    benchmark_ids = f.read().splitlines()

try:
    df = pd.read_csv(full_df_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    print(f"Loaded {len(df)} rows of data from {full_df_path}")
except FileNotFoundError:
    print(f"Couldn't find cached CSV at {full_df_path}. Loading data from DynamoDB.")
    chunks = []

    for benchmark_id in benchmark_ids:
        chunk = load_benchmark(benchmark_id)
        if chunk is not None:
            chunks.append(chunk)
    df = pd.concat(chunks)
    df.to_csv(full_df_path, index=False)

Loaded 184536 rows of data from ./data/all-data.csv


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184536 entries, 0 to 184535
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              184536 non-null  datetime64[ns]
 1   batch_size             184536 non-null  int64         
 2   image_size             184536 non-null  int64         
 3   backend                184536 non-null  object        
 4   salad_machine_id       184536 non-null  object        
 5   images                 184536 non-null  object        
 6   qr_data                184536 non-null  object        
 7   prompt                 184536 non-null  object        
 8   steps                  184536 non-null  int64         
 9   qrGenTime              184536 non-null  float64       
 10  imageGenTime           184536 non-null  float64       
 11  gpu                    184536 non-null  object        
 12  vram                   184536 non-null  floa

In [4]:
df.head()

,timestamp,batch_size,image_size,backend,salad_machine_id,images,qr_data,prompt,steps,qrGenTime,imageGenTime,gpu,vram,totalTime,gpu_performance_score
0,2023-11-13 21:17:34.592,1,512,stable-fast-qr-code,67c6c009-1018-8a5f-8658-dd8162c753bd,['https://salad-benchmark-assets.download/sd-1...,https://salad.com,leafy green salad,15,0.502758,111.598356,GTX 1650,4.0,112.106,1650.1
1,2023-11-13 21:17:34.592,1,512,stable-fast-qr-code,67c6c009-1018-8a5f-8658-dd8162c753bd,['https://salad-benchmark-assets.download/sd-1...,https://salad.com,leafy green salad,15,0.509855,100.297959,GTX 1650,4.0,100.813,1650.1
2,2023-11-13 21:19:45.664,1,512,stable-fast-qr-code,67c6c009-1018-8a5f-8658-dd8162c753bd,['https://salad-benchmark-assets.download/sd-1...,https://salad.com,leafy green salad,15,0.513511,101.023498,GTX 1650,4.0,101.543,1650.1
3,2023-11-13 21:21:56.736,1,512,stable-fast-qr-code,67c6c009-1018-8a5f-8658-dd8162c753bd,['https://salad-benchmark-assets.download/sd-1...,https://salad.com,leafy green salad,15,0.510402,100.517272,GTX 1650,4.0,101.033,1650.1
4,2023-11-13 21:24:07.808,1,512,stable-fast-qr-code,67c6c009-1018-8a5f-8658-dd8162c753bd,['https://salad-benchmark-assets.download/sd-1...,https://salad.com,leafy green salad,15,0.502009,100.377556,GTX 1650,4.0,100.885,1650.1


In [5]:
def get_data_for_gpu(gpu: str):
    gpu_df = df[df["gpu"] == gpu]
    return gpu_df

gpu_classes = get_gpu_classes()
print(f"{len(gpu_classes)} GPU classes found")

23 GPU classes found


In [8]:
# Get the total number of images generated. Can be determined by summing batch_size in all rows
total_images = df["batch_size"].sum()

# total with size 512
total_images_512 = df[df["image_size"] == 512]["batch_size"].sum()

# total with size 768
total_images_768 = df[df["image_size"] == 768]["batch_size"].sum()

print(f"Total images generated: {total_images}")
print(f"Total images generated with size 512: {total_images_512}")
print(f"Total images generated with size 768: {total_images_768}")

Total images generated: 443179
Total images generated with size 512: 234862
Total images generated with size 768: 208317


In [15]:
gpu_short_names = list(map(lambda item: shorten_gpu_name(item["name"]), gpu_classes))
def sort_gpu_list(gpu_list):
    # Helper function to extract numerical value and Ti status from GPU model name
    def extract_gpu_info(model):
        # Extract numerical value
        num_value = int(''.join(filter(str.isdigit, model)))
        # Check for 'Ti' version
        is_ti = 1 if model.endswith('Ti') else 0
        return num_value + is_ti

    # Sort the list using the helper function
    gpu_list.sort(key=lambda model: extract_gpu_info(model))

    return gpu_list

gpu_short_names = sort_gpu_list(gpu_short_names)
for name in gpu_short_names:
    num_results = len(get_data_for_gpu(name))
    print(f"{name}: {num_results} results")

GTX 1050 Ti: 1189 results
GTX 1060: 2579 results
GTX 1070: 4403 results
GTX 1650: 2617 results
GTX 1660: 5920 results
GTX 1660: 5920 results
RTX 2060: 9962 results
RTX 2070: 9827 results
RTX 2080: 9620 results
RTX 2080 Ti: 9637 results
RTX 3050: 13348 results
RTX 3060: 4800 results
RTX 3060 Ti: 9675 results
RTX 3070: 9372 results
RTX 3070 Ti: 9256 results
RTX 3080: 9591 results
RTX 3080 Ti: 9397 results
RTX 3090: 9564 results
RTX 3090 Ti: 9600 results
RTX 4070: 9635 results
RTX 4070 Ti: 9608 results
RTX 4080: 9600 results
RTX 4090: 9601 results


In [45]:
def summarize_gpu(gpu: str):
    gpu_df = get_data_for_gpu(gpu)
    total_rows_in_group = len(gpu_df)
    sum_of_batch_sizes = gpu_df["batch_size"].sum()
    print("##################################################")
    print(f"\n{gpu}: {total_rows_in_group} rows, {sum_of_batch_sizes} images\n")

    # Group by backend and show how many were completed for each backend
    backend_counts = gpu_df.groupby("backend")


    for backend in backend_counts.groups.keys():
        total_rows_in_group = len(backend_counts.groups[backend])
        sum_of_batch_sizes = backend_counts.get_group(backend)["batch_size"].sum()
        print(f"{backend}: {total_rows_in_group} rows, {sum_of_batch_sizes} images")


    print("\nBy Image Size")

    # Group by image size and show how many were completed for each image size, for each backend
    image_size_counts = gpu_df.groupby("image_size")

    for image_size in image_size_counts.groups.keys():
        for backend in backend_counts.groups.keys():
            try:
              total_rows_for_backend_and_image_size = len(image_size_counts.get_group(image_size).groupby("backend").groups[backend])
              sum_of_batch_sizes = image_size_counts.get_group(image_size).groupby("backend").get_group(backend)["batch_size"].sum()
              print(f"{image_size}x{image_size} {backend}: {total_rows_for_backend_and_image_size} rows, {sum_of_batch_sizes} images")
            except KeyError:
              print(f"{image_size}x{image_size} {backend}: 0 rows, 0 images")

    print("\nBy Batch Size")

    # Group by batch size and show how many were completed for each batch size, for each backend
    batch_size_counts = gpu_df.groupby("batch_size")

    for batch_size in batch_size_counts.groups.keys():
        print()
        for image_size in image_size_counts.groups.keys():
          print()
          for backend in backend_counts.groups.keys():
              selection = gpu_df[gpu_df["batch_size"] == batch_size]
              selection = selection[selection["image_size"] == image_size]
              selection = selection[selection["backend"] == backend]
              total_rows_for_backend_and_image_size = len(selection)
              sum_of_batch_sizes = selection["batch_size"].sum()
              avg_total_time = selection["totalTime"].mean()
              print(f"{batch_size} {image_size}x{image_size} {backend}: {total_rows_for_backend_and_image_size} rows, {sum_of_batch_sizes} images, {avg_total_time} seconds")

for gpu in gpu_short_names:
    summarize_gpu(gpu)

##################################################

GTX 1050 Ti: 1189 rows, 2489 images

a1111: 330 rows, 622 images
comfy: 702 rows, 1633 images
sdnext: 154 rows, 231 images
stable-fast-qr-code: 3 rows, 3 images

By Image Size
512x512 a1111: 282 rows, 559 images
512x512 comfy: 662 rows, 1573 images
512x512 sdnext: 117 rows, 194 images
512x512 stable-fast-qr-code: 3 rows, 3 images
768x768 a1111: 48 rows, 63 images
768x768 comfy: 40 rows, 60 images
768x768 sdnext: 37 rows, 37 images
768x768 stable-fast-qr-code: 0 rows, 0 images

By Batch Size


1 512x512 a1111: 119 rows, 119 images, 31.696151260504205 seconds
1 512x512 comfy: 209 rows, 209 images, 62.02645933014354 seconds
1 512x512 sdnext: 80 rows, 80 images, 57.699137500000006 seconds
1 512x512 stable-fast-qr-code: 3 rows, 3 images, 247.241 seconds

1 768x768 a1111: 33 rows, 33 images, 165.0228484848485 seconds
1 768x768 comfy: 20 rows, 20 images, 86.80255000000001 seconds
1 768x768 sdnext: 37 rows, 37 images, 125.76637837837835 secon